In [1]:
import torch
import random
import numpy as np
from SnakeAI import SnakeGameAI, Direction, Point
from collections import deque
from helper import plot

from model import Linear_QNet, QTrainer

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
MAX_MEMORY=100_000
BATCH_SIZE = 1000
LR = 0.001

In [3]:
import torch
import random
import numpy as np
from collections import deque
from SnakeAI import SnakeGameAI, Direction, Point
from model import Linear_QNet, QTrainer

MAX_MEMORY = 100_000
BATCH_SIZE = 1000
LR = 0.001

class Agent:

    def __init__(self):
        self.n_games = 0
        self.epsilon = 0 # randomness
        self.gamma = 0.9 # discount rate
        self.memory = deque(maxlen=MAX_MEMORY) # popleft()
        self.model = Linear_QNet(11, 256, 3)
        self.trainer = QTrainer(self.model, lr=LR, gamma=self.gamma)


    def get_state(self, game):
        head = game.snake[0]
        point_l = Point(head.x - 20, head.y)
        point_r = Point(head.x + 20, head.y)
        point_u = Point(head.x, head.y - 20)
        point_d = Point(head.x, head.y + 20)
        
        dir_l = game.direction == Direction.LEFT
        dir_r = game.direction == Direction.RIGHT
        dir_u = game.direction == Direction.UP
        dir_d = game.direction == Direction.DOWN

        state = [
            # Danger straight
            (dir_r and game.is_collision(point_r)) or 
            (dir_l and game.is_collision(point_l)) or 
            (dir_u and game.is_collision(point_u)) or 
            (dir_d and game.is_collision(point_d)),

            # Danger right
            (dir_u and game.is_collision(point_r)) or 
            (dir_d and game.is_collision(point_l)) or 
            (dir_l and game.is_collision(point_u)) or 
            (dir_r and game.is_collision(point_d)),

            # Danger left
            (dir_d and game.is_collision(point_r)) or 
            (dir_u and game.is_collision(point_l)) or 
            (dir_r and game.is_collision(point_u)) or 
            (dir_l and game.is_collision(point_d)),
            
            # Move direction
            dir_l,
            dir_r,
            dir_u,
            dir_d,
            
            # Food location 
            game.food.x < game.head.x,  # food left
            game.food.x > game.head.x,  # food right
            game.food.y < game.head.y,  # food up
            game.food.y > game.head.y  # food down
            ]

        return np.array(state, dtype=int)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # popleft if MAX_MEMORY is reached

    def train_long_memory(self):
        if len(self.memory) > BATCH_SIZE:
            mini_sample = random.sample(self.memory, BATCH_SIZE) # list of tuples
        else:
            mini_sample = self.memory

        states, actions, rewards, next_states, dones = zip(*mini_sample)
        self.trainer.train_step(states, actions, rewards, next_states, dones)
        #for state, action, reward, nexrt_state, done in mini_sample:
        #    self.trainer.train_step(state, action, reward, next_state, done)

    def train_short_memory(self, state, action, reward, next_state, done):
        self.trainer.train_step(state, action, reward, next_state, done)

    def get_action(self, state):
        # random moves: tradeoff exploration / exploitation
        self.epsilon = 80 - self.n_games
        final_move = [0,0,0]
        if random.randint(0, 200) < self.epsilon:
            move = random.randint(0, 2)
            final_move[move] = 1
        else:
            state0 = torch.tensor(state, dtype=torch.float)
            prediction = self.model(state0)
            move = torch.argmax(prediction).item()
            final_move[move] = 1

        return final_move


def train():
    plot_scores = []
    plot_mean_scores = []
    total_score = 0
    record = 0
    agent = Agent()
    game = SnakeGameAI()
    while True:
        # get old state
        state_old = agent.get_state(game)

        # get move
        final_move = agent.get_action(state_old)

        # perform move and get new state
        reward, done, score = game.play_step(final_move)
        state_new = agent.get_state(game)

        # train short memory
        agent.train_short_memory(state_old, final_move, reward, state_new, done)

        # remember
        agent.remember(state_old, final_move, reward, state_new, done)

        if done:
            # train long memory, plot result
            game.reset()
            agent.n_games += 1
            agent.train_long_memory()

            if score > record:
                record = score
                agent.model.save()

            print('Game', agent.n_games, 'Score', score, 'Record:', record)

            plot_scores.append(score)
            total_score += score
            mean_score = total_score / agent.n_games
            plot_mean_scores.append(mean_score)

if __name__ == '__main__':
    train()

C:\Users\user\Music\Linear Games\model.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  state = torch.tensor(state, dtype=torch.float)


Game 1 Score 0 Record: 0
Game 2 Score 0 Record: 0
Game 3 Score 0 Record: 0
Game 4 Score 0 Record: 0
Game 5 Score 1 Record: 1
Game 6 Score 0 Record: 1
Game 7 Score 0 Record: 1
Game 8 Score 0 Record: 1
Game 9 Score 0 Record: 1
Game 10 Score 0 Record: 1
Game 11 Score 0 Record: 1
Game 12 Score 1 Record: 1
Game 13 Score 0 Record: 1
Game 14 Score 0 Record: 1
Game 15 Score 0 Record: 1
Game 16 Score 1 Record: 1
Game 17 Score 1 Record: 1
Game 18 Score 0 Record: 1
Game 19 Score 1 Record: 1
Game 20 Score 0 Record: 1
Game 21 Score 0 Record: 1
Game 22 Score 0 Record: 1
Game 23 Score 0 Record: 1
Game 24 Score 1 Record: 1
Game 25 Score 0 Record: 1
Game 26 Score 1 Record: 1
Game 27 Score 0 Record: 1
Game 28 Score 0 Record: 1
Game 29 Score 0 Record: 1
Game 30 Score 0 Record: 1
Game 31 Score 1 Record: 1
Game 32 Score 1 Record: 1
Game 33 Score 0 Record: 1
Game 34 Score 0 Record: 1
Game 35 Score 1 Record: 1
Game 36 Score 1 Record: 1
Game 37 Score 0 Record: 1
Game 38 Score 2 Record: 2
Game 39 Score 1 Recor

NameError: name 'quit' is not defined